<a href="https://colab.research.google.com/github/JVCarmich0959/CSC228/blob/main/Jackie's_T0_Experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.0 MB/s eta 0:00:00

In [ ]:
import transformers
from transformers import pipeline
import numpy as np
import pandas as pd
from datasets import DatasetDict
from datasets import load_dataset
import pprint
from typing import List


# Loading in my Dataset:




In [ ]:
# Load the CSV file from the URL using pandas
url = "https://raw.githubusercontent.com/JVCarmich0959/CSC228/main/Sample_Questions.csv"
df = pd.read_csv(url)

 # Defining a "QuestionAnsweringSystem":

In the "init" function, it initializes two pipelines from the transformers library:

* "bigscience/T0_3B" model - This pipeline is designed to generate text in response to a given prompt. 


* "question-answering"(distilbert-base-cased-distilled-squad): used for extracting answers from a given context and question.


My theory was that by harnessing both the powers of extractive, and generative pipelines I would end up with a question answering system can **leverage the strengths of each pipeline and provide a more robust and accurate answers to a wider range of questions.** For example, the "text2text-generation" pipeline can be used to generate multiple possible answers to a more complex question, while the "question-answering" pipeline can be used to extract a specific answer to a more factual question.

TL;DR: Supervised + Unsupervised = Better Model

In [ ]:
class QuestionAnsweringSystem:
    def __init__(self):
        self.text_generation = pipeline("text2text-generation", model="bigscience/T0_3B")
        self.extractive_qa = pipeline("question-answering")

    def generate_answers(self, questions, contexts):
        responses = []
        for question, context in zip(questions, contexts):
            prompt = f"Context: {context}\nAnswer the question: {question}"
            response = self.text_generation(prompt, max_length=600, do_sample=True, num_return_sequences=3, temperature=0.7)

            # Select the best answer (this example simply chooses the first one)
            answer = response[0]['generated_text'].replace(prompt, '').strip()
            responses.append(answer)
        return responses

In [ ]:
questions = df['question'].tolist() # converting the data from the DataFrame into a list
contexts = df['context'].tolist()

qa_system = QuestionAnsweringSystem() # intialize our class
responses = qa_system.generate_answers(questions, contexts) # generate a list of responses

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


# Let's take a look at our dataset :

In [ ]:
pprint.pprint(questions[0])
pprint.pprint(responses[0]) # The response is logical however not completely accurate...

'What courses are offered at this community college?'
'Agriculture and Natural Resources'


In [ ]:
pprint.pprint(questions[1])
pprint.pprint(responses[1]) # This is actually also kind of mostly correct and probably the best way to apply to Wayne

'How do I apply to this community college?'
('Apply online at '
 'https://www2.cfnc.org/exclusive.html#/applicationlogin/WaynCC/Undergrad/WaynCCUndergradApp, '
 'or in person at the admissions office .')


In [ ]:
pprint.pprint(questions[2])
pprint.pprint(responses[2]) # kind of Vague but not completely inaccurate

'What financial aid options are available at this community college?'
('The Financial Aid office offers various private and non-profit scholarships '
 'each year.')


In [ ]:
pprint.pprint(questions[3])
pprint.pprint(responses[3]) 

# If...I could be honest and I had to choose one...yes...this is outside of the buisness department 
# The most effectively helpful student resource on the campus.

('What kind of support services are available to students at this community '
 'college?')
'The Academic Skills Center'


In [ ]:
pprint.pprint(questions[4])
pprint.pprint(responses[4]) # The only real way to engage with the campus is through its community organized clubs this is not an 
# uninformed answer...
# If you want to network and get more involved with the college joining one of the 16 active clubs is your 
# best chance.

' What is campus life like at this community college?'
'You can get involved in clubs and organizations (C&O)'


In [ ]:
pprint.pprint(questions[5])
pprint.pprint(responses[5]) 

#mmmmmm no

('What transfer opportunities are available to students at this community '
 'college?')
'Wayne Community College'


In [ ]:
pprint.pprint(questions[6])
pprint.pprint(responses[6]) # This is true but I'm not sure it answers the question very well...

'Can I take courses online at this community college?'
('All classes here at WCC use our Moodle system / Open LMS to some degree and '
 'a vast majority of courses use other online resources')


In [ ]:
pprint.pprint(questions[7])
pprint.pprint(responses[7]) # Once again not inaccurate

'What are the qualifications of the faculty at this community college?'
('Wayne Community College is accredited by the Southern Association of '
 'Colleges and Schools Commission on Colleges (SACSCOC) to award associate '
 'degrees. Degree-granting institutions also may offer credentials such as '
 'certificates and diplomas at approved degree levels')


In [ ]:
pprint.pprint(questions[8])
pprint.pprint(responses[8]) # This is pretty good!

'What kind of facilities does this community college have?'
('state-of-the art classrooms, laboratories, offices, and a model child care '
 'center')


In [ ]:
pprint.pprint(questions[9])
pprint.pprint(responses[9]) # This has to do with the quality of my dataset so I won't fault the model here.

'What kind of community outreach programs does this community college have?'
('It has funding for scholarships, equipment, and unique learning '
 'opportunities for who they consider to be deserving students or otherwise '
 'profitable investments')


Overall the Class object we used to test my theory by using both pipelines is very accurate...but I think it isn't dynamic enough. It seems that having both the generative and extractive pipelines works well enough but needs some serious tweaks.

TL;DR: NOT BAD! NEEDS IMPROVEMENT.

# Some other Questions

Below I am using just the T0_3B pipeline by itself. To gain an understanding of extractive vs. generative pipelines for NLP tasks.I like the performance of the zero-shot classifiers but it's too eco-unfriendly to use liberally.

These questions are about Wayne Community and contexts were sourced from:
* https://www.waynecc.edu/catalog/
* The student planner that we frisbee at new or potential students.

Let's look:


In [ ]:
text_generation = pipeline("text2text-generation", model="bigscience/T0_3B")

# Define a question and context
question_1 = "How do I apply to the nursing program?"
context_1 = """Students must present evidence of good physical 
and mental health (including required immunizations and health 
screening) upon admission. Details on health requirements are 
listed in the admission packet. Applicants must provide evidence 
of current certification in an American Heart Association (AHA) 
Basic Life Support (BLS) Cardio Pulmonary Resuscitation (CPR) and 
Automated External Defibrillator (AED) course for Healthcare Professionals
that includes hands-on skill demonstration. Upon admission to the nursing
program, students are required to submit to a drug screening. Applicants 
will be charged a fee to offset the cost of the drug screening.
Results of the criminal background check and drug screen may determine
if a student is eligible to enter and/or remain in all clinical agencies 
to progress within the program. If a clinical site denies a student 
placement in their facility, the student would be unable to complete
the required clinical component of the course; therefore, the 
student will be withdrawn from all nursing courses and will not be 
allowed to progress in the program."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=95, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Applicants must present evidence of good physical and mental health '
 '(including required immunizations and health screening) upon admission. '
 'Details on health requirements are listed in the admission packet. '
 'Applicants must provide evidence of current certification in an American '
 'Heart Association (AHA) Basic Life Support (BLS) Cardio Pulmonary '
 'Resuscitation (CPR) and Automated External Defibrillator (AED) course for '
 'Healthcare professionals. Upon admission to the nursing program, students '
 'are required to submit to a drug screening.')


Already I really like what I am seeing so far
It cuts the unecessary fat of of your context and 
does a great job of giving pertinent information.

Giving potential nursing students the quick of their academic pathway is essential for their success. 

In [ ]:
# Define a question and context
question_1 = "How do I get a Student ID?"
context_1 = """All WCC curriculum students are required to obtain and maintain 
an identification card and a parking decal. You can obtain your Parking Decal 
and Student ID once you have completed the registration process and paid  your 
tuition. Once this has been done, visit Student Activities located near the 
Student Lounge in the Wayne Learning Center with your receipt.
Each student receives only one free ID and parking decal. After registering
and paying for your classes, bring your receipt to the Student Activities office 
located near the Student Lounge in the Wayne Learning Center to get your student 
ID card and parking decal.Damaged or lost IDs will be replaced at a cost of $5.00 
per ID and additional parking decals are $2.00. Temporary parking passes can be 
obtained in the Student Activities office.Student ID cards are validated each semester
in the Business Office or Student Activities Office upon payment of fees.Call the Student 
Activities office at (919) 739-6740 or (919) 739-6742 for office hours and other information."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=95, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('After registering and paying for your classes, bring your receipt to the '
 'Student Activities office located near the Student Lounge in the Wayne '
 'Learning Center to get your student ID card and parking decal. Each student '
 'receives only one free ID and parking decal. Damaged or lost IDs will be '
 'replaced at a cost of $5.00 per ID and additional parking decals are $2.00. '
 'Temporary parking passes can be obtained in the Student Activities '
 'office.Student ID cards are validated each semester in the Business Office '
 'or Student Activities Office upon payment of fees.')


I get this question daily. How do I get a College Identification card? Who do I have to kill to get a Parking Pass? I need a temporary parking pass...etc. It's pretty repetitive...this does a great job of handling those kind of general questions.

In [ ]:
# Define a question and context
question_1 = "I lost my ID How can I get a replacement?"
context_1 = """All WCC curriculum students are required to obtain and maintain 
an identification card and a parking decal. You can obtain your Parking Decal 
and Student ID once you have completed the registration process and paid  your 
tuition. Once this has been done, visit Student Activities located near the 
Student Lounge in the Wayne Learning Center with your receipt.
Each student receives only one free ID and parking decal. After registering
and paying for your classes, bring your receipt to the Student Activities office 
located near the Student Lounge in the Wayne Learning Center to get your student 
ID card and parking decal.Damaged or lost IDs will be replaced at a cost of $5.00 
per ID and additional parking decals are $2.00. Temporary parking passes can be 
obtained in the Student Activities office.Student ID cards are validated each semester
in the Business Office or Student Activities Office upon payment of fees.Call the Student 
Activities office at (919) 739-6740 or (919) 739-6742 for office hours and other information."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

'Replacements will be issued for $5.00 fee and additional parking decals $2.00'


In [ ]:
# Define a question and context
question_1 = "Where can I get a temporary parking pass?"
context_1 = """All WCC curriculum students are required to obtain and maintain 
an identification card and a parking decal. You can obtain your Parking Decal 
and Student ID once you have completed the registration process and paid  your 
tuition. Once this has been done, visit Student Activities located near the 
Student Lounge in the Wayne Learning Center with your receipt.
Each student receives only one free ID and parking decal. After registering
and paying for your classes, bring your receipt to the Student Activities office 
located near the Student Lounge in the Wayne Learning Center to get your student 
ID card and parking decal.Damaged or lost IDs will be replaced at a cost of $5.00 
per ID and additional parking decals are $2.00. Temporary parking passes can be 
obtained in the Student Activities office.Student ID cards are validated each semester
in the Business Office or Student Activities Office upon payment of fees.Call the Student 
Activities office at (919) 739-6740 or (919) 739-6742 for office hours and other information."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length = 15 ,do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Student Activities office The Student Activities office is located near the '
 'Student Lounge in the Wayne Learning Center  aplications office')


This is working perfectly really well! It even provided directions...Now if only they new where the Wayne Learning Center was? (There usually standing in it)

In [ ]:
# Define a question and context
question_1 = "Tell me about the Accounting Program."
context_1 = """Get the skills necessary for employment 
and growth in the accounting and finance profession. Be 
a part of one of the hottest career fields with projected 
job growth of 18 percent. Accounting offers you a great 
career with a world of opportunities and flexibility.
Course work may include accounting, finance, ethics, 
business law, computer applications, financial planning, 
insurance, marketing, real estate, selling, and taxation. 
Related skills are developed through the study of communications, 
computer applications, financial analysis, critical thinking skills, and ethics.
Graduates should qualify for entry-level accounting and finance positions in many 
types of organizations including accounting firms, small businesses, manufacturing 
firms, banks, hospitals, school systems, and governmental agencies.
This degree can be earned through traditional seated classes, totally online, 
or hybrid classes."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=95, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Be a part of one of the hottest career fields with projected job growth of '
 '18 percent. Accounting offers you a great career with a world of '
 'opportunities and flexibility. Course work may include accounting, finance, '
 'ethics, business law, computer applications, financial planning, insurance, '
 'marketing, real estate, selling, and taxation. Related skills are developed '
 'through the study of communications, computer applications, financial '
 'analysis, critical thinking skills, and ethics. Graduates should qualify for '
 'entry-level accounting and finance positions in many types of organizations '
 'including accounting firms, small businesses, manufacturing firms, banks, '
 'hospitals, school systems, and governmental agencies. This degree can be '
 'earned through traditional seated classes, totally online, or hybrid '
 'classes.')


OH NO! Looks like we have a bunch of jibberish it may be due to the way we fraised our question.

In [ ]:
# Define a question and context
question_1 = "Who is the president of the college?"
context_1 = """Dr. Patricia A. “Patty” Pfeiffer began her career at Wayne Community College in November 1994 as a nursing instructor. She rose to Nursing Department chair, dean of the Allied Health and Public Services Division, chief of staff and vice president of institutional effectiveness, then vice president of academic and student services.

She took on the role of interim president of the college starting on June 24, 2021 and will continue to Dec. 31, 2021. She becomes the college’s ninth executive on Jan. 1, 2022.

Pfeiffer has been recognized in her various roles at the college. She received WCC’s top award for faculty members, the George E. Wilson Excellence in Teaching Award in 2007 and was also a finalist for the NC State Board of Community College’s Excellence in Teaching Award. In 2015, the college’s unit of the Association of Educational Office Professionals (AEOP) and the North Carolina District 12 AEOP presented Pfeiffer with their Administrator of the Year Awards.

Prior to employment with the college, Pfeiffer worked as a labor and delivery nurse.

Before attending college, Pfeiffer served in the United States Army as a parachute rigger and airborne paratrooper for six years. Her first duty station was Fort Lewis, Washington where she supported the 2nd and 75th Infantry Ranger Battalion. In addition, she was stationed in Taegu, Korea, and Fort Bragg, NC.

Pfeiffer earned her doctorate in higher education and adult learning from Walden University.  She also holds a master’s degree in administration from Central Michigan University, a master’s degree in nursing education from East Carolina University, a bachelor’s degree in nursing from East Carolina University, and an associate degree in nursing from Wayne Community College.

Pfeiffer is married to Tim, and they have two adult daughters, two grandchildren, and one son-in-law."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=95, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Patty Pfeiffer began her career at Wayne Community College in November 1994 '
 'as a nursing instructor. She rose to Nursing Department chair, dean of the '
 'Allied Health and Public Services Division, chief of staff and vice '
 'president of institutional effectiveness, then vice president of academic '
 'and student services. She took on the role of interim president beginning on '
 "June 24, 2021 and will continue to Dec. 31, 2021. She becomes the college's "
 'ninth executive on Jan. 1, 2022. Pfeiffer has been recognized in her various '
 "roles at the college. She received WCC's top award for faculty members, the "
 'George E. Wilson Excellence in Teaching Award in 2007 and was also a '
 "finalist for the NC State Board of Community College's Excellence in "
 'Teaching Award. In 2015, the college’s unit of the Association of '
 'Educational Office Professionals (AEOP) and the North Carolina District 12 '
 'AEOP presented Pfeiffer with their Administrator of the Year Awards. Pri

In [ ]:
# Define a question and context
question_1 = "Does the Bookstore accept returns?"
context_1 = """Wayne Community College maintains a bookstore for the convenience of students.
Course supplies and required textboks (new, used, e-text, rentals), gifts, clothing, electronics,
and graduation attire may be purchased in the bookstore. Visit the website at www.wayneccshop.com
for fast and easy textbok information and on-line ordering. Hours of operation are Monday, Tuesday,
and Thursday from 8:00 a.m. to 4:00 p.m., Wednesday from 8:00 a.m. to 5:00 p.m. and Friday from 8:00 a.m.
to 1:00 p.m. Extended hours are available at the start of each semester and other times as needed. Summer hours will vary.
Current hours are posted on the bookstore's website. The store's return policy allows a student seven
days from the start of the semester or two days from the purchase date if after the refund deadline.
All materials must be in the same condition as purchaed(shrink wrap intact, codes unopened and/or scratched,
packaging unopened). General merchandise may be returned up to 30 days from the purchase in original 
condition. Laptops are not returnable if opened and must be returned within 14 days of purchase. Refunds
are provided by original tender only(cash provided for check transacions after 10 buisness days).
Students may sell books back at any time during the year except at the beginning of each semester while 
financial aid is open for charging. """

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600,  do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

'No'


without specifying the 'min_length' parameter the model was a little tart. But still correct.

In [ ]:
# Define a question and context
question_1 = "Where is the lost and found?"
context_1 = """Lost and found articles are kept in the Campus Police and Security Office(1st floor, Wayne Learning 
Center, Room 195). Articles that are not reclaimed will be disposed of or turned over to appropriate agencies
after 30 days. """

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

'The Campus Police and Security Office'


In [ ]:
# Define a question and context
question_1 = "How do I access Moodle?"
context_1 = """
Click the Moodle link on the right side of the page.
Log In information is on the Moodle Site.
Once logged in, click the title of the course(s)
Read the welcome announcements and complete initial
assignmets and the enrollment verification quiz.
For additional information, please visit www.waynecc.edu/de
If you would like to preview a WCC Moodle course site:
Go to moodle.waynecc.edu. Type "wccguest" for your user name
and "wccguest" for your password.
Click the tile of the WCC Moodle Demo Course.
Click the "Logout" button to exit.
"""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=15, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Log In information is on the Moodle Site. Once logged in, click on the title '
 'of the course(s)')


In [ ]:
# Define a question and context
question_1 = "Help! I can't find the computer lab!"
context_1 = """The Open Computer lab is located in Magnolia 215 and is available to WCC students
with a valid ID card (obtained in the student activities office). It contains 50 PCs, printers and a
scanner. All instructional software offered by the College, along with Internet connectivity, is available.
Consult the open computer lab website at http://www.waynecc.edu/online-services/technology-at-wayne/open-computer-lab/ 
for hours of operation. """

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Open Computer lab is located in Magnolia 215 and is available to WCC '
 'students with a valid ID card (obtained in the student activities office)')


In [ ]:
# Define a question and context
question_1 = "How many clubs are at the College?"
context_1 = """There are 16 active  clubs and organizations that have
indicated they will be active in the 2022-2023 academic year - pending 
official approval. Students should contact the advisors to learn more about
the student clubs and organizations:
Associate Degree Student Nurse Association Level I
Associate Degree Student Nurse ASsociation Level II
Criminal Justice Club
Dental Assisting Club
Dental Hygiene Club
Fishing Club
Forestry Club
Honors Club
Human Services Club
Multicultural Club
Pharmacy Technology Club
Phi Theta Kappa
Practical Nursing Club
Wayne Community College Society of American Foresters
Wayne Community College Future Leaders of the World (WCC FLOW)
Wayne COmmunity College Association of Nursing Students."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

'16'


In [ ]:
# Define a question and context
question_1 = "I am a new student how do I register?"
context_1 = """Registration Guide
New Students :
(Degree/Diploma/Certificate Seeking)
If you’re a brand new student, you will register for classes
through your academic advisor after completing the initial 
application steps:
*Apply online at: https://apply.cfnc.org/application/WaynCC/Undergrad/WaynCCUndergradApp?logoUrl=https://xroadsapi.cfnc.org/application/WaynCC/Undergrad/WaynCCUndergradApp/exclusiveLogo
*Activate your MyBisonID Account: https://www.waynecc.edu/mybisonid/
*Apply for Financial Aid: https://studentaid.gov/h/apply-for-aid/fafsa school code(008216)
*Submit your Transcripts to : wcc-admissions@waynecc.edu
*Take Placement Tests (only if required)
*Meet your Academic Advisor: After the application process, you will be assigned an academic advisor whom you can contact to register for courses. The name of your assigned advisor and their contact information can be viewed in the Self-Service tool located through the Okta Portal.
Inside Self-Service, navigate to: Student Planning -> Plan your Degree & Register for Classes -> Advising tab
Current Students (Degree/Diploma/Certificate Seeking):
So you have at least one semester under your belt and you’re 
ready to hit Priority Registration for your next semester. 
Great! Don’t forget that all students are required to meet 
with their advisor before registration. Also, remember that 
if for some reason you missed Priority Registration, you can 
always register in any of the Open Registration periods.
Search for Classes: Check the Course Schedules page to see what
courses are available to you for the next semester.Register 
through Self-Service When Priority and Open Registration are
active, you use Self-Service to register for classes. You will
be contacted by the college with information about when you can 
register for classes. You also can use Self-Service to view your 
class schedule, GPA, transcripts, grades, and more.
Login Help Visit the My Bison ID page if you are having trouble 
logging into Self-Service or any other services in the Okta Portal.
Tuition, Fees, and Refund Policy Visit the Tuition and Fees page 
to find information about tuition and fees, along with a Net Price Calculator.
Information about our tuition refund policy can be found on 
the Refund and Disbursements page."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=200, min_length =50, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('No, I am not a new student. I am trying to register for courses for next '
 'semester. What do I do? There are different ways you can register for '
 'courses, so you should get more information from the university. In '
 'addition, there can be various reasons why you are unable to register '
 'electronically suflete: financial aid , health care , etc')


In [ ]:
# Define a question and context
question_1 = "If I am a current student how do I register?"
context_1 = """Registration Guide
New Students :
(Degree/Diploma/Certificate Seeking)
If you’re a brand new student, you will register for classes
through your academic advisor after completing the initial 
application steps:
*Apply online at: https://apply.cfnc.org/application/WaynCC/Undergrad/WaynCCUndergradApp?logoUrl=https://xroadsapi.cfnc.org/application/WaynCC/Undergrad/WaynCCUndergradApp/exclusiveLogo
*Activate your MyBisonID Account: https://www.waynecc.edu/mybisonid/
*Apply for Financial Aid: https://studentaid.gov/h/apply-for-aid/fafsa school code(008216)
*Submit your Transcripts to : wcc-admissions@waynecc.edu
*Take Placement Tests (only if required)
*Meet your Academic Advisor: After the application process, you will be assigned an academic advisor whom you can contact to register for courses. The name of your assigned advisor and their contact information can be viewed in the Self-Service tool located through the Okta Portal.
Inside Self-Service, navigate to: Student Planning -> Plan your Degree & Register for Classes -> Advising tab
Current Students (Degree/Diploma/Certificate Seeking):
So you have at least one semester under your belt and you’re 
ready to hit Priority Registration for your next semester. 
Great! Don’t forget that all students are required to meet 
with their advisor before registration. Also, remember that 
if for some reason you missed Priority Registration, you can 
always register in any of the Open Registration periods.
Search for Classes: Check the Course Schedules page to see what
courses are available to you for the next semester.Register 
through Self-Service When Priority and Open Registration are
active, you use Self-Service to register for classes. You will
be contacted by the college with information about when you can 
register for classes. You also can use Self-Service to view your 
class schedule, GPA, transcripts, grades, and more.
Login Help Visit the My Bison ID page if you are having trouble 
logging into Self-Service or any other services in the Okta Portal.
Tuition, Fees, and Refund Policy Visit the Tuition and Fees page 
to find information about tuition and fees, along with a Net Price Calculator.
Information about our tuition refund policy can be found on 
the Refund and Disbursements page."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length= 15, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

("You 'll register through your assigned academic advisor . All students are "
 'required to meet with their advisor before registration. Also, remember that '
 'if for some reason you missed Priority Registration, you can always register '
 'in any of the Open Registration periods. I would like someone to contact me '
 'about the classes I saw on the website.')


In [ ]:
# Define a question and context
question_1 = "What is the school code?"
context_1 = """Registration Guide
New Students :
(Degree/Diploma/Certificate Seeking)
If you’re a brand new student, you will register for classes
through your academic advisor after completing the initial 
application steps:
*Apply online at: https://apply.cfnc.org/application/WaynCC/Undergrad/WaynCCUndergradApp?logoUrl=https://xroadsapi.cfnc.org/application/WaynCC/Undergrad/WaynCCUndergradApp/exclusiveLogo
*Activate your MyBisonID Account: https://www.waynecc.edu/mybisonid/
*Apply for Financial Aid: https://studentaid.gov/h/apply-for-aid/fafsa school code(008216)
*Submit your Transcripts to : wcc-admissions@waynecc.edu
*Take Placement Tests (only if required)
*Meet your Academic Advisor: After the application process, you will be assigned an academic advisor whom you can contact to register for courses. The name of your assigned advisor and their contact information can be viewed in the Self-Service tool located through the Okta Portal.
Inside Self-Service, navigate to: Student Planning -> Plan your Degree & Register for Classes -> Advising tab
Current Students (Degree/Diploma/Certificate Seeking):
So you have at least one semester under your belt and you’re 
ready to hit Priority Registration for your next semester. 
Great! Don’t forget that all students are required to meet 
with their advisor before registration. Also, remember that 
if for some reason you missed Priority Registration, you can 
always register in any of the Open Registration periods.
Search for Classes: Check the Course Schedules page to see what
courses are available to you for the next semester.Register 
through Self-Service When Priority and Open Registration are
active, you use Self-Service to register for classes. You will
be contacted by the college with information about when you can 
register for classes. You also can use Self-Service to view your 
class schedule, GPA, transcripts, grades, and more.
Login Help Visit the My Bison ID page if you are having trouble 
logging into Self-Service or any other services in the Okta Portal.
Tuition, Fees, and Refund Policy Visit the Tuition and Fees page 
to find information about tuition and fees, along with a Net Price Calculator.
Information about our tuition refund policy can be found on 
the Refund and Disbursements page."""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

'Wayne Community College'


In [ ]:
# Define a question and context
question_1 = "I need something to drink where are the vending machines?"
context_1 = """Vending machines are located on the first floor of WLC (snacks/
sandwhiches/drinks), 2nd floor of Magnolia (snack/drinks), 1st floor of Hocutt
(snacks/drinks), 2nd floor of Walnut (snacks/drinks), ourside the north entrance 
to Pine(drinks only), at the Wayne Works Center, and at the Aviation Classroom
Building(drinks only). You are also welcome to visit the Bison Bistro in the 
Wayne Learning Center. """

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('vending machines are located on the 1st floor of WLC (snacks/ '
 'sandwhiches/drinks), 2nd floor of Magnolia (snack/drinks), 1st floor of '
 'Hocutt (snack/drinks), 2nd floor of Walnut (snack/drinks), ourside the north '
 'entrance to Pine(drinks only), at the Wayne Works Center, and at the '
 'Aviation Classroom Building(drinks only)')


In [ ]:
# Define a question and context
question_1 = "What time does the Library open?"
context_1 = """
The Dr. Clyde A. Erwin, Jr. Library 
provides resources to assist students, faculty, and staff in their research and actively supports the College 
mission by the ongoing development of a print and electronic collection devoted to enhancing the curriculum. 
Library services are offered to traditional students and distance learners, as well as to the general public 
of Wayne County.
Offerings of the Library include the following:
Over 36,000 circulating print books supplemented by nearly 400,000 ebooks
Over 80,000 streaming videos
Thousands of fully-searchable online magazines, newspapers and journals available for in-library and remote use
32 desktop computers and headphones for research and homework with WCC username login or guest pass
The WCC Single Search discovery service, providing access to NCLIVE along with 62 other research databases 
for article, ebook and digital video resourcesCustomizable information literacy or group study sessions in the 
Library Literacy Lab as well as presentations given off-site or available in Moodle course pages Services for 
creating scans, photocopying, laser printing and wireless printing Reference and research assistance in person, 
by telephone or e-mail 4 group study pods seating 6-8 people and a larger literacy lab offering many wall- and 
table-mounted outlets for reservation or use on a first-come, first-served basis
Hours of operation are:
Monday through Thursday - 7:45 a.m. to 7:00 p.m.
Friday - 7:45 a.m. to 1:00 p.m.
(Hours may vary during Summer Term.)
The Dr. Clyde A. Erwin, Jr. Library is located on the third floor of the Wayne Learning Center building. 
For more information, call (919) 739-6891 or visit online at the library website:www.waynecc.edu/library.
 """

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('Monday through Thursday - 7:45 a.m. to 7:00 p.m. Friday - 7:45 a.m. to 1:00 '
 'p.m.')


In [ ]:
# Define a question and context
question_1 = "How do I search for articles in the Library?"
context_1 = """
The Dr. Clyde A. Erwin, Jr. Library 
provides resources to assist students, faculty, and staff in their research and actively supports the College 
mission by the ongoing development of a print and electronic collection devoted to enhancing the curriculum. 
Library services are offered to traditional students and distance learners, as well as to the general public 
of Wayne County.
Offerings of the Library include the following:
Over 36,000 circulating print books supplemented by nearly 400,000 ebooks
Over 80,000 streaming videos
Thousands of fully-searchable online magazines, newspapers and journals available for in-library and remote use
32 desktop computers and headphones for research and homework with WCC username login or guest pass
The WCC Single Search discovery service, providing access to NCLIVE along with 62 other research databases 
for article, ebook and digital video resources Customizable information literacy or group study sessions in the 
Library Literacy Lab as well as presentations given off-site or available in Moodle course pages Services for 
creating scans, photocopying, laser printing and wireless printing Reference and research assistance in person, 
by telephone or e-mail 4 group study pods seating 6-8 people and a larger literacy lab offering many wall- and 
table-mounted outlets for reservation or use on a first-come, first-served basis
Hours of operation are:
Monday through Thursday - 7:45 a.m. to 7:00 p.m.
Friday - 7:45 a.m. to 1:00 p.m.
(Hours may vary during Summer Term.)
The Dr. Clyde A. Erwin, Jr. Library is located on the third floor of the Wayne Learning Center building. 
For more information, call (919) 739-6891 or visit online at the library website:www.waynecc.edu/library.
 """

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('The WCC Single Search discovery service, providing access to NCLIVE along '
 'with 62 other research databases for article, ebook and digital video '
 'resources')


I wanted to see how effective this pipeline was for emergencies where it may be important for students and staff to be informed.

In [ ]:
# Define a question and context
question_1 = "Where do I go in a tornado warning?"
context_1 = """
Severe Weather Alert
In the event that a severe weather warning presents an immediate threat to WCC facilities, the campus will be alerted by the Bison Alert system and the following plan will be activated. If time permits, disconnect all electrical and computer equipment and close all windows before you leave your area.
Azalea, Dogwood, Holly, Magnolia, Pine, Spruce, and Walnut Buildings:  Occupants should immediately proceed to the first and second floors of their respective buildings and assemble in the interior rooms, hallways, or stairs away from any outside doors and windows and wait for further instructions from Campus Police.
Ash Building:  Occupants should immediately proceed to the first floor of the building and assemble on the first floor in the Storage Room 126, 103, or Break Room 118.  Stay away from glass walls and windows and wait for further instructions from Campus Police.
Aviation Building: All individuals should immediately proceed to the Aviation classroom building interior hallway, rest rooms, kitchenette, offices, etc.  Stay away from outside doors and windows and wait for further instructions from Campus Police.
Hocutt Building: Occupants should immediately proceed to the ﬁrst ﬂoor and assemble in the interior rooms, hallways, or stairs away from any outside doors and windows and wait for further instructions from Campus Police.
Cedar and Oak Buildings: All individuals should immediately proceed to the interior hallway or rooms with no outside doors or windows.  Stay as far as possible away from windows and exterior doors  Individuals should wait there for further instructions from Campus Police.
WLC Building: All individuals on the first floor should immediately proceed to the Moffatt Auditorium and move to the first row and fill in from the middle of the row in the center or inside out on the right and left wings so everyone can find a seat as quickly as possible.  If the Moffatt Auditorium is already at capacity, go to the interior hallways, restrooms, break rooms, kitchenettes, office areas, and other interior spaces with no outside doors or windows on the first and second floors, and wait for further instructions from Campus Police.  On the second and third floors, avoid outside doors and windows, including those facing the atrium.
Elsewhere on Campus:  Individuals in service buildings or elsewhere on the grounds should seek immediate shelter in the nearest brick building or lay flat in a ditch or other low-lying area if time does not permit getting inside a building.
Everyone is to remain in the safe area(s) until notified by Campus Police that all is clear. 
"""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=15, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('WLC Building: All individuals on the first floor should immediately proceed '
 'to the Moffatt Auditorium and move to the first row and fill in from the '
 'middle of the row in the center or inside out on the right and left wings so '
 'everyone can find a seat as quickly as possible')


Sound advice but I'm sure someone would want to know the specifics...

In [ ]:
# Define a question and context
question_1 = "How will the campus be alerted in case of severe weather warnings?"
context_1 = """
Severe Weather Alert
In the event that a severe weather warning presents an immediate threat to WCC facilities, the campus will be alerted by the Bison Alert system and the following plan will be activated. If time permits, disconnect all electrical and computer equipment and close all windows before you leave your area.
Azalea, Dogwood, Holly, Magnolia, Pine, Spruce, and Walnut Buildings:  Occupants should immediately proceed to the first and second floors of their respective buildings and assemble in the interior rooms, hallways, or stairs away from any outside doors and windows and wait for further instructions from Campus Police.
Ash Building:  Occupants should immediately proceed to the first floor of the building and assemble on the first floor in the Storage Room 126, 103, or Break Room 118.  Stay away from glass walls and windows and wait for further instructions from Campus Police.
Aviation Building: All individuals should immediately proceed to the Aviation classroom building interior hallway, rest rooms, kitchenette, offices, etc.  Stay away from outside doors and windows and wait for further instructions from Campus Police.
Hocutt Building: Occupants should immediately proceed to the ﬁrst ﬂoor and assemble in the interior rooms, hallways, or stairs away from any outside doors and windows and wait for further instructions from Campus Police.
Cedar and Oak Buildings: All individuals should immediately proceed to the interior hallway or rooms with no outside doors or windows.  Stay as far as possible away from windows and exterior doors  Individuals should wait there for further instructions from Campus Police.
WLC Building: All individuals on the first floor should immediately proceed to the Moffatt Auditorium and move to the first row and fill in from the middle of the row in the center or inside out on the right and left wings so everyone can find a seat as quickly as possible.  If the Moffatt Auditorium is already at capacity, go to the interior hallways, restrooms, break rooms, kitchenettes, office areas, and other interior spaces with no outside doors or windows on the first and second floors, and wait for further instructions from Campus Police.  On the second and third floors, avoid outside doors and windows, including those facing the atrium.
Elsewhere on Campus:  Individuals in service buildings or elsewhere on the grounds should seek immediate shelter in the nearest brick building or lay flat in a ditch or other low-lying area if time does not permit getting inside a building.
Everyone is to remain in the safe area(s) until notified by Campus Police that all is clear. 
"""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=15, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('The Bison Alert system. This alert will be activated. If time permits, '
 'disconnect all electrical and computer equipment and close all windows '
 'before you leave your area. Azalea, Dogwood, Holly, Magnolia, PineJubiläum, '
 'Spruce, Walnut Buildings')


In [ ]:
# Define a question and context
question_1 = "How long should people remain in safe areas during a tornado warning?"
context_1 = """
Severe Weather Alert
In the event that a severe weather warning presents an immediate threat to WCC facilities, the campus will be alerted by the Bison Alert system and the following plan will be activated. If time permits, disconnect all electrical and computer equipment and close all windows before you leave your area.
Azalea, Dogwood, Holly, Magnolia, Pine, Spruce, and Walnut Buildings:  Occupants should immediately proceed to the first and second floors of their respective buildings and assemble in the interior rooms, hallways, or stairs away from any outside doors and windows and wait for further instructions from Campus Police.
Ash Building:  Occupants should immediately proceed to the first floor of the building and assemble on the first floor in the Storage Room 126, 103, or Break Room 118.  Stay away from glass walls and windows and wait for further instructions from Campus Police.
Aviation Building: All individuals should immediately proceed to the Aviation classroom building interior hallway, rest rooms, kitchenette, offices, etc.  Stay away from outside doors and windows and wait for further instructions from Campus Police.
Hocutt Building: Occupants should immediately proceed to the ﬁrst ﬂoor and assemble in the interior rooms, hallways, or stairs away from any outside doors and windows and wait for further instructions from Campus Police.
Cedar and Oak Buildings: All individuals should immediately proceed to the interior hallway or rooms with no outside doors or windows.  Stay as far as possible away from windows and exterior doors  Individuals should wait there for further instructions from Campus Police.
WLC Building: All individuals on the first floor should immediately proceed to the Moffatt Auditorium and move to the first row and fill in from the middle of the row in the center or inside out on the right and left wings so everyone can find a seat as quickly as possible.  If the Moffatt Auditorium is already at capacity, go to the interior hallways, restrooms, break rooms, kitchenettes, office areas, and other interior spaces with no outside doors or windows on the first and second floors, and wait for further instructions from Campus Police.  On the second and third floors, avoid outside doors and windows, including those facing the atrium.
Elsewhere on Campus:  Individuals in service buildings or elsewhere on the grounds should seek immediate shelter in the nearest brick building or lay flat in a ditch or other low-lying area if time does not permit getting inside a building.
Everyone is to remain in the safe area(s) until notified by Campus Police that all is clear. 
"""

# Format the prompt
prompt_1 = f"Context: {context_1}\nAnswer the question: {question_1}"

# Generate the answer
response_1 = text_generation(prompt_1, max_length=600, min_length=15, do_sample=True, num_return_sequences=1)

# Extract the answer from the response
answer_1 = response_1[0]['generated_text'].replace(prompt_1, '').strip()

# Print the answer
pprint.pprint(answer_1)

('1 hour unless otherwise instructed by Campus police. After an order is given '
 'students must return their classrooms')


Wow...

# Zero-Shot Classification 

Out performs traditional supervised learning models (But that is too be expected) 

In an extractive question answering pipeline, the model is trained on a labeled dataset where each example consists of a question and a corresponding paragraph or document from which the answer needs to be extracted. During training, the model learns to identify the relevant passages from the given context that contain the answer to the question. This is typically achieved by training the model to predict the start and end positions of the answer span within the given context.

In contrast, zero-shot classification requires minimal labeled data and can generalize to new categories that were not present during training! 



# For demo purposes

Often times, you have to present the functionality of your model to someone who doesn't understand the background details. For this you can use the gradio library (I found this to be really helpful on the hugging face website)

The Gradio library allows you to create interactive user interfaces (UI) for your machine learning models, making it easy for anyone to experiment with your model's inputs and visualize the outputs. Gradio can be used to create UIs for a variety of model types, such as text generation, image classification, object detection, and more.

In [ ]:
import gradio as gr

text_generation = pipeline("text2text-generation", model="bigscience/T0_3B")

def generate_answer(question, context):
    prompt = f"Context: {context}\nAnswer the question: {question}"
    response = text_generation(prompt, max_length=600, do_sample=True, num_return_sequences=1)
    answer = response[0]['generated_text'].replace(prompt, '').strip()
    return answer

def gradio_interface(question, context):
    return generate_answer(question, context)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=["text", "text"],
    outputs="text",
    example = []
   
)
iface.launch()

/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:43: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'example': []}
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>